# 土炮股票分析系統

這個範例只是將上課所講過的所有例子串在一起，提供一個簡單的概念，讓大家知道一個簡單的分析系統大概可以怎麼做。

例子裡面有很多不足的地方，請自行思考如何修改。

In [1]:
# 載入需要的模組
import ffn
import pandas as pd
import pandas_datareader.data as web

from datetime import datetime
import numpy as np

## 前面定義好的一些函式

In [2]:
# 取得公司資料
def get_companies(ex = "NASDAQ"):
    template = "http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange={}&render=download"
    url = template.format(ex)
    return pd.read_csv(url)

In [3]:
# 計算波動率
def volatility(symbol, startdate):
    try:
        df = web.DataReader(symbol, 'yahoo', startdate)
        dailyRet = df['Close'].pct_change()
        return dailyRet.std()
    except Exception as e:
        print("Symbol: ", symbol, "==> Error: ", e.args)
        return 10000.0

In [4]:
# 計算 MaxDD
def DrawDownAnalysis(cumRet):
    dd_series = ffn.core.to_drawdown_series(cumRet)
    dd_details = ffn.core.drawdown_details(dd_series)
    return dd_details['drawdown'].min(), dd_details['days'].max()

## 抓取公司資料

In [5]:
data = get_companies()

In [6]:
data.head()

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
0,PIH,"1347 Property Insurance Holdings, Inc.",7.35,4.398803e+07,n/a,2014,Finance,Property-Casualty Insurers,http://www.nasdaq.com/symbol/pih,NaN
1,TURN,180 Degree Capital Corp.,2.02,6.286556e+07,n/a,n/a,Finance,Finance/Investors Services,http://www.nasdaq.com/symbol/turn,NaN
2,FLWS,"1-800 FLOWERS.COM, Inc.",10.25,6.624007e+08,n/a,1999,Consumer Services,Other Specialty Stores,http://www.nasdaq.com/symbol/flws,NaN
3,FCCY,1st Constitution Bancorp (NJ),18.25,1.473351e+08,n/a,n/a,Finance,Savings Institutions,http://www.nasdaq.com/symbol/fccy,NaN
4,SRCE,1st Source Corporation,51.64,1.339342e+09,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/srce,NaN


In [7]:
# 看一下有幾筆資料
len(data.index)

3270

In [8]:
# 也可以這樣看
data.shape

(3270, 10)

In [9]:
# 作為範例取出前十筆來做就好了，因為太多筆沒辦法在課堂上示範...
# 另外連續頻繁抓資料，會被 Yahoo Finance 視為攻擊，之後會抓不到資料。
companylist = data['Symbol'][0:10].tolist()

## 波動率選股

我們通常只抓一次股市資料，然後存到資料庫裏面去，之後就從自己的資料庫裏面撈數據出來分析。

這個地方因為為了搭配先前講過的例子，所以只是把例子裡面的做法放到 Function 裡面來。

須注意這樣的作法只是上課 Demo 用。

In [10]:
results = []

for symbol in companylist:
    vo = volatility(symbol, datetime(2016, 1, 1))
    results.append((vo, symbol))
    
results.sort()

results

Symbol:  VNET ==> Error:  ('Unable to read URL: https://query1.finance.yahoo.com/v7/finance/download/VNET?period1=1451577600&period2=1512230399&interval=1d&events=history&crumb=GI%5Cu002FXjWqgbL9',)
Symbol:  EGHT ==> Error:  ('Unable to read URL: https://query1.finance.yahoo.com/v7/finance/download/EGHT?period1=1451577600&period2=1512230399&interval=1d&events=history&crumb=%5Cu002FM13lubPNuo',)


[(0.014320224444026822, 'SRCE'),
 (0.016155762484458602, 'FCCY'),
 (0.017658247825673733, 'CAFD'),
 (0.018123379281956464, 'JOBS'),
 (0.021464920969773663, 'PIH'),
 (0.022796214867284018, 'TWOU'),
 (0.023158994218157095, 'TURN'),
 (0.023621409952326564, 'FLWS'),
 (10000.0, 'EGHT'),
 (10000.0, 'VNET')]

In [11]:
# 選出波動率最小的前五檔股票
computer_selected = np.array(results)[:, 1][:5].tolist()
computer_selected

['SRCE', 'FCCY', 'CAFD', 'JOBS', 'PIH']

# 加入自選股

In [12]:
# 上面是電腦幫忙選股...
# 底下是自己因為看新聞、聽小道消息等等的靈機一閃後，想看看看回測狀況好不好的股票
# 它只是一個簡單的 list 會被一起放進去回測看看
self_selected = ['TSLA', 'GOOG', 'YHOO', 'MSFT', 'AAPL']

In [13]:
candidates = computer_selected + self_selected

In [14]:
# 放到 set 中的重複性資料只會被保留一份
candidates = set(candidates)

In [15]:
# 這是刪掉重複資料後的候選名單
candidates = list(candidates)

# 回測

In [35]:
# 利用策略產生的持有部位資訊，計算底下四個指標來判斷投資績效
# sharpe ratio: 判斷報酬的好壞跟穩定度，數值越大越好
# maxdd: maximum drawdown, 最糟糕的狀況會賠幾 %
# maxddd: maximum drawdown duration, 低於上一次最高報酬的天數
# cumRet[-1]: 最後賺的 % 數

def indicators(df):
    dailyRet = df['Close'].pct_change()
    if np.all(df['positions']==0):
        return 0.0, 0.0, 0.0, 0.0
    excessRet = (dailyRet - 0.04/252)[df['positions'] == 1]
    SharpeRatio = np.sqrt(252.0)*np.mean(excessRet)/np.std(excessRet)
    
    cumRet = np.cumprod(1+excessRet)
    
    maxdd, maxddd = DrawDownAnalysis(cumRet)
    
    return SharpeRatio, maxdd, maxddd, cumRet[-1]

In [17]:
# 這是我們的策略的部分
# 主要只是要算出進出的訊號 signals 跟何時持有部位 positions
# 底下是一個突破系統的範例

def breakout(df):
    # Donchian Channel
    df['20d_high'] = np.round(pd.Series.rolling(df['Close'], window=20).max(), 2)
    df['10d_low'] = np.round(pd.Series.rolling(df['Close'], window=10).min(), 2)

    has_position = False
    df['signals'] = np.zeros(df['Close'].shape)
    for t in range(2, df['signals'].size):
        if df['Close'][t] > df['20d_high'][t-1]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['Close'][t] < df['10d_low'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum()
    return df

In [18]:
def RSI_8020(df):
    df['RSI'] = talib.RSI(df['Close'].values)
    
    has_position = False
    df['signals'] = np.zeros(np.size(df['Close']))
    for t in range(2, df['signals'].size):
        if df['RSI'][t-1] < 20:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['RSI'][t-1] > 80:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum()

    return df

In [19]:
def RSI_7030(df):
    df['RSI'] = talib.RSI(df['Close'].values)
    
    has_position = False
    df['signals'] = np.zeros(np.size(df['Close']))
    for t in range(2, df['signals'].size):
        if df['RSI'][t-1] < 30:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['RSI'][t-1] > 70:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum()
    return df

In [20]:
def apply_strategy(strategy, df):
    return strategy(df)

In [21]:
# 先把所有股票資料抓下來，放到字典上面去
# 因為我們在這裡沒有使用資料庫，所以用字典來做存放
all_data = {}

for symbol in candidates:
    try:
        all_data[symbol] = web.DataReader(symbol, 'yahoo', datetime(2016,1,1))
    except Exception as e:
        all_data[symbol] = pd.DataFrame()
        print("Error accessing symbol: ", symbol, "==>", e.args)

Error accessing symbol:  SRCE ==> ('Unable to read URL: https://query1.finance.yahoo.com/v7/finance/download/SRCE?period1=1451577600&period2=1512230399&interval=1d&events=history&crumb=L1TZ34CK%5Cu002FZc',)
Error accessing symbol:  YHOO ==> ('Unable to read URL: https://query1.finance.yahoo.com/v7/finance/download/YHOO?period1=1451577600&period2=1512230399&interval=1d&events=history&crumb=woXyFXgul18',)
Error accessing symbol:  JOBS ==> ('Unable to read URL: https://query1.finance.yahoo.com/v7/finance/download/JOBS?period1=1451577600&period2=1512230399&interval=1d&events=history&crumb=6bhGExi3jb%5Cu002F',)


In [23]:
import talib

In [25]:
# 計算各支股票的回測結果
results = []

for symbol in candidates:
    if all_data[symbol].empty:
        continue
    apply_strategy(breakout, all_data[symbol])
    SharpeRatio, maxdd, maxddd, finalRet = indicators(all_data[symbol])
    results.append((SharpeRatio, maxdd, maxddd, finalRet, symbol))

results

[(2.6417396805507143, -0.089302619785146087, 102, 1.4330050968937249, 'CAFD'),
 (2.7432716475936747, -0.074378594911961748, 118, 1.4525720402497386, 'GOOG'),
 (4.0853683871960884, -0.041393948519331425, 59, 2.2528791875638259, 'AAPL'),
 (3.4154063434378514, -0.041053355557252424, 70, 1.6871203416763842, 'MSFT'),
 (2.2845054416061092, -0.10355155203187105, 158, 1.6088144873029671, 'PIH'),
 (4.5181382107717711, -0.088670347018806717, 77, 2.8811512035541433, 'TSLA'),
 (2.8362235577624091, -0.065837620029339128, 254, 1.6561195902576635, 'FCCY')]

In [36]:
# 計算各支股票的回測結果
results = []

strategies = [breakout, RSI_7030, RSI_8020]

for symbol in candidates:
    if all_data[symbol].empty:
            continue
    for strategy in strategies:
        apply_strategy(strategy, all_data[symbol])
        SharpeRatio, maxdd, maxddd, finalRet = indicators(all_data[symbol])
        results.append((SharpeRatio, maxdd, maxddd, finalRet, symbol, strategy.__name__))

results

[(2.6417396805507143,
  -0.089302619785146087,
  102,
  1.4330050968937249,
  'CAFD',
  'breakout'),
 (0.39655599512014778,
  -0.21701704255802612,
  148,
  1.0294188438405492,
  'CAFD',
  'RSI_7030'),
 (0.66871222730868229,
  -0.20097259641798138,
  150,
  1.1449210372842593,
  'CAFD',
  'RSI_8020'),
 (2.7432716475936747,
  -0.074378594911961748,
  118,
  1.4525720402497386,
  'GOOG',
  'breakout'),
 (10.805005633339752,
  -0.0062513344778371005,
  3,
  1.1462679986680928,
  'GOOG',
  'RSI_7030'),
 (0.0, 0.0, 0.0, 0.0, 'GOOG', 'RSI_8020'),
 (4.0853683871960884,
  -0.041393948519331425,
  59,
  2.2528791875638259,
  'AAPL',
  'breakout'),
 (1.8550716451444915,
  -0.086421685311413254,
  61,
  1.1649344832998494,
  'AAPL',
  'RSI_7030'),
 (0.0, 0.0, 0.0, 0.0, 'AAPL', 'RSI_8020'),
 (3.4154063434378514,
  -0.041053355557252424,
  70,
  1.6871203416763842,
  'MSFT',
  'breakout'),
 (2.0764905207064683,
  -0.088991685137862375,
  42,
  1.1103673589094081,
  'MSFT',
  'RSI_7030'),
 (0.0, 0.0

In [26]:
# 排序股票，取 Sharpe Ratio 高的前幾名當標的
sorted(results, reverse=True)

[(4.5181382107717711, -0.088670347018806717, 77, 2.8811512035541433, 'TSLA'),
 (4.0853683871960884, -0.041393948519331425, 59, 2.2528791875638259, 'AAPL'),
 (3.4154063434378514, -0.041053355557252424, 70, 1.6871203416763842, 'MSFT'),
 (2.8362235577624091, -0.065837620029339128, 254, 1.6561195902576635, 'FCCY'),
 (2.7432716475936747, -0.074378594911961748, 118, 1.4525720402497386, 'GOOG'),
 (2.6417396805507143, -0.089302619785146087, 102, 1.4330050968937249, 'CAFD'),
 (2.2845054416061092, -0.10355155203187105, 158, 1.6088144873029671, 'PIH')]

In [37]:
results_df = pd.DataFrame(results, columns=['sharpe','MaxDrawDown','MaxDrawDownDuration','returns','symbol','strategy'])

In [38]:
results_df

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,symbol,strategy
0,2.641740,-0.089303,102.0,1.433005,CAFD,breakout
1,0.396556,-0.217017,148.0,1.029419,CAFD,RSI_7030
2,0.668712,-0.200973,150.0,1.144921,CAFD,RSI_8020
3,2.743272,-0.074379,118.0,1.452572,GOOG,breakout
4,10.805006,-0.006251,3.0,1.146268,GOOG,RSI_7030
5,0.000000,0.000000,0.0,0.000000,GOOG,RSI_8020
6,4.085368,-0.041394,59.0,2.252879,AAPL,breakout
7,1.855072,-0.086422,61.0,1.164934,AAPL,RSI_7030
8,0.000000,0.000000,0.0,0.000000,AAPL,RSI_8020
9,3.415406,-0.041053,70.0,1.687120,MSFT,breakout


In [39]:
results_df.sort_values('MaxDrawDown',ascending=False)

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,symbol,strategy
20,0.000000,0.000000,0.0,0.000000,FCCY,RSI_8020
11,0.000000,0.000000,0.0,0.000000,MSFT,RSI_8020
5,0.000000,0.000000,0.0,0.000000,GOOG,RSI_8020
8,0.000000,0.000000,0.0,0.000000,AAPL,RSI_8020
4,10.805006,-0.006251,3.0,1.146268,GOOG,RSI_7030
9,3.415406,-0.041053,70.0,1.687120,MSFT,breakout
6,4.085368,-0.041394,59.0,2.252879,AAPL,breakout
18,2.836224,-0.065838,254.0,1.656120,FCCY,breakout
3,2.743272,-0.074379,118.0,1.452572,GOOG,breakout
7,1.855072,-0.086422,61.0,1.164934,AAPL,RSI_7030
